In [9]:
import Data 
import  pandas as pd
import copy
import numpy as np
import os 
from statsmodels.tsa.api import SimpleExpSmoothing


In [2]:
# download data 
stock_ticker = ['VTI','AGG','DBC','^VIX']
names1 = ['VTI_daily.csv','AGG_daily.csv','DBC_daily.csv','VIX_daily.csv']
names2 = ['VTI_weekly.csv','AGG_weekly.csv','DBC_weekly.csv','VIX_weekly.csv']

stock_daily_price  = Data.data_download(stock_ticker,names1,
                                        start_date='2015-01-05',
                                        end_date='2021-03-22',
                                        frequence='1d')
stock_weekly_price  = Data.data_download(stock_ticker,names1,
                                        start_date='2015-01-05',
                                        end_date='2021-03-22',
                                        frequence='1wk')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
# calculate the return 
stock_daily_return = Data.stock_return(stock_daily_price)
stock_weekly_return = Data.stock_return(stock_daily_price)

# calcualte the stock movement 
stock_daily_movement = Data.stock_movement(stock_return=stock_daily_return)
stock_weekly_movement = Data.stock_movement(stock_return=stock_weekly_return)

# smoothing ohlc data 
pre_stock_dailyprice = Data.data_smoothing(stock_daily_price)
pre_stock_weeklyprice = Data.data_smoothing(stock_weekly_price)


/Users/baihaohuang/Desktop/TestForLSTM/Data.py:43: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  i[0:] = move
/Users/baihaohuang/Desktop/TestForLSTM/Data.py:43: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  i[0:] = move
/Users/baihaohuang/Desktop/TestForLSTM/Data.py:43: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  i[0:] = move
/Users/baihaohuang/Desktop/TestForLSTM/Data.py:43: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  i[0:] = move
/Users/baihaohuang/Desktop/TestForLSTM/Data.py:43: F

In [4]:
# feature engineering 
feature_daily = Data.feature_engineering(pre_stock_dailyprice)
feature_weekly = Data.feature_engineering(pre_stock_weeklyprice)

/Users/baihaohuang/opt/anaconda3/envs/untitled/lib/python3.7/site-packages/pandas_ta/overlap/ema.py:19: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  close[:length - 1] = npNaN
/Users/baihaohuang/opt/anaconda3/envs/untitled/lib/python3.7/site-packages/pandas_ta/overlap/ema.py:19: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  close[:length - 1] = npNaN
/Users/baihaohuang/opt/anaconda3/envs/untitled/lib/python3.7/site-packages/pandas_ta/overlap/ema.py:19: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  close[:length - 1] = npNaN
/Users/baihaohuang/opt/anaconda3/envs/untitled/lib/python3.7/site-packages/pandas_ta/ove

In [5]:
# Min-Max normalizing the feature 
pre_feature_daily = Data.data_normalization(feature_daily,save_scaler=False)
pre_feature_weekly = Data.data_normalization(feature_weekly,save_scaler=False)

In [6]:
# combine the y and x 

# predict price 
regression_daily = Data.data_combination(stock_daily_price,feature_daily,'regression')
regression_weekly = Data.data_combination(stock_weekly_price,feature_weekly,'regression')
regression_daily,scaler_daily  = Data.data_normalization(regression_daily,save_scaler=True)
regression_weekly,scaler_weekly = Data.data_normalization(regression_weekly,save_scaler=True)

# predict movement 
classification_daily = Data.data_combination(stock_daily_movement, feature_daily ,'classification')
classification_weekly = Data.data_combination(stock_weekly_movement, feature_weekly,'classification')

In [7]:
# shift the data 
regression_daily_shift = Data.data_shift(regression_daily)
regression_weekly_shift = Data.data_shift(regression_weekly)
classification_daily_shift = Data.data_shift(classification_daily)
classification_weekly_shift = Data.data_shift(classification_weekly)

In [13]:
# save the file 
document_name1 = 'processed_data/Daily_Regression'
document_name2 = 'processed_data/Weekly_Regression'
document_name3 = 'processed_data/Daily_Classification'
document_name4 = 'processed_data/Weekly_Classification'

file_name1 = ['rd_VTI','rd_AGG','rd_DBC','rd_VIX']
file_name2 = ['rw_VTI','rw_AGG','rw_DBC','rw_VIX']
file_name3 = ['cd_VTI','cd_AGG','cd_DBC','cd_VIX']
file_name4 = ['cw_VTI','cw_AGG','cw_DBC','cw_VIX']

Data.data_writer(regression_daily_shift,document_name1,file_name1)
Data.data_writer(regression_weekly_shift,document_name2,file_name2)
Data.data_writer(classification_daily_shift,document_name3,file_name3)
Data.data_writer(classification_weekly_shift,document_name4,file_name4)

In [14]:
# save the scaler 
current = os.getcwd() + '/scaler'

# convert into array 
scaler_daily = np.asarray(scaler_daily)
scaler_weekly = np.asarray(scaler_weekly)

# save as npy
np.save(current + '/Daily.npy',scaler_daily) 
np.save(current + '/Weekly.npy',scaler_weekly) 